In [1]:
from chatbot import Chat,reflections,multiFunctionCall
import wikipedia

In [2]:
def whoIs(query,sessionID="general"):
    try:
        return wikipedia.summary(query)
    except:
        for newquery in wikipedia.search(query):
            try:
                return wikipedia.summary(newquery)
            except:
                pass
    return "I don't know about "+query

In [3]:
from keras.preprocessing.image import img_to_array
import imutils
import cv2
from keras.models import load_model
import numpy as np
import playsound
# parameters for loading data and images
detection_model_path = 'Emotion/haarcascade_files/haarcascade_frontalface_default.xml'
emotion_model_path = 'Emotion/models/_mini_XCEPTION.102-0.66.hdf5'

# hyper-parameters for bounding boxes shape
# loading models
face_detection = cv2.CascadeClassifier(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)
EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised",
 "neutral"]

def emo(query,sessionID="general"):
    cv2.namedWindow('your_face')
    camera = cv2.VideoCapture(0)

    while True:
        frame = camera.read()[1]
        frame = imutils.resize(frame,width=300)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)
        canvas = np.zeros((250, 300, 3), dtype="uint8")
        frameClone = frame.copy()
        if len(faces) > 0:
            faces = sorted(faces, reverse=True,
            key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
            (fX, fY, fW, fH) = faces
            roi = gray[fY:fY + fH, fX:fX + fW]
            roi = cv2.resize(roi, (64, 64))
            roi = roi.astype("float") / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            
            preds = emotion_classifier.predict(roi)[0]
            emotion_probability = np.max(preds)
            label = EMOTIONS[preds.argmax()]

            ee = []
            percent = []
            for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                ee.append(emotion)
                percent.append(prob)
            mp = percent.index(max(percent))

        cv2.imshow('your_face', frameClone)
        break
    camera.release()
    cv2.destroyAllWindows()
    try:
        return ee[mp] 
    except:
        return "I cannot see your face."

Using TensorFlow backend.


In [4]:
call = multiFunctionCall({"whoIs":whoIs,"emo":emo})
firstQuestion="Hi, how are you?"
Chat("Example.template", reflections,call=call).converse(firstQuestion)

Hi, how are you?
> I am great
 why you feel great 
> Because you are working perfectly
Is that the real reason?
> yep
You seem quite sure.
> can you tell how i feel 
neutral
> can you tell how i feel 
happy
> tell me about ai
In computer science,  artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines, in contrast to the natural intelligence displayed by humans.  Colloquially, the term "artificial intelligence" is often used to describe machines (or computers) that mimic "cognitive" functions that humans associate with the human mind, such as "learning" and "problem solving".As machines become increasingly capable, tasks considered to require "intelligence" are often removed from the definition of AI, a phenomenon known as the AI effect. A quip in Tesler's Theorem says "AI is whatever hasn't been done yet." For instance, optical character recognition is frequently excluded from things considered to be AI, having become a routine te

In [5]:
Chat("Example.template", reflections,call=call).converse(firstQuestion)

Hi, how are you?
> Do you remember my name?
No, I do not know your name, Can you please tell me your name?
> I am Pratik
Thank you pratik
> Do you remember my name?
Yes I do pratik
> quit
Thank you for talking with me.
